In [34]:
#importing dependencies and setting app token
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime
import time
import requests
import sqlite3
from api_keys import MyAppToken
import json

#checking the day of the month and printing the result, this is used to filter the dataframe later
today = date.today()
daynum = today.strftime("%d")
month = today.strftime("%m")
year = today.strftime("%Y")
day = int(daynum) - 7
print(f"Today: {today} \n\tyear: {year} \n\tmonth: {month} \n\tday of month: {daynum} ")

Today: 2020-05-03 
	year: 2020 
	month: 05 
	day of month: 03 


# Aggregations data example
Doesn't have credit score or rates though

In [35]:
baseUrl = 'https://ffiec.cfpb.gov/v2/data-browser-api/view/aggregations?'
filters = [
    'states=MD'
    , 'years=2018'
    , 'actions_taken=5,6'
    , 'races=White,Asian,Joint'
] 

# Using list comprehensions, create the url filter:
finalUrl = baseUrl + ''.join("&%s" % x for x in filters)
print(finalUrl)

try:    
    response = (requests.get(finalUrl).text)
    response_json = json.loads(response)
    time.sleep(0.5)
    print(response_json)

except Exception as e:
    raise e

https://ffiec.cfpb.gov/v2/data-browser-api/view/aggregations?&states=MD&years=2018&actions_taken=5,6&races=White,Asian,Joint
{'parameters': {'year': '2018', 'state': 'MD', 'actions_taken': '5,6', 'races': 'White,Asian,Joint'}, 'aggregations': [{'count': 716, 'sum': 254350000.0, 'actions_taken': '6', 'races': 'Asian', 'state': 'MD', 'year': '2018'}, {'count': 243, 'sum': 94325000.0, 'actions_taken': '6', 'races': 'Joint', 'state': 'MD', 'year': '2018'}, {'count': 165, 'sum': 47595000.0, 'actions_taken': '5', 'races': 'Joint', 'state': 'MD', 'year': '2018'}, {'count': 7505, 'sum': 2323145000.0, 'actions_taken': '6', 'races': 'White', 'state': 'MD', 'year': '2018'}, {'count': 679, 'sum': 190835000.0, 'actions_taken': '5', 'races': 'Asian', 'state': 'MD', 'year': '2018'}, {'count': 5237, 'sum': 1241115000.0, 'actions_taken': '5', 'races': 'White', 'state': 'MD', 'year': '2018'}]}


# CSV data - 2018
One thing to note is that each year potentially has a different schema

- [API documentation](https://cfpb.github.io/hmda-platform/#data-browser-api-csv-example)
- [Modified LAR data schema](https://ffiec.cfpb.gov/documentation/2020/modified-lar-schema/)
- [Github LAR file spec](https://github.com/cfpb/hmda-platform/blob/master/docs/v2/spec/2018_File_Spec_LAR.csv)
- [Not sure - but def. API documentation](https://github.com/cfpb/hmda-platform/blob/master/docs/v2/api/public-api.md)


### Usage notes
1. Can use either `nationwide` or specify one or more states
2. For state, use 2 character state name
3. For msamd use a 5 digit integer
4. For counties use 5 digit county code


### Filtering
Very difficult since the field names, endpoint names, attributes are all "similar, yet different" across the CSV endpoint, the JSON endpoint, the downloaded CSV, and the schema

* [Tests on github offer some insight](https://github.com/cfpb/hmda-platform/blob/645df9bda48635ac66c89e0cae380967b62f4423/newman/tests/data-browser/data-browser-api-config.json)



### Github
* [Main HDMA repository](https://github.com/cfpb/hmda-platform)
* [HDMA's github repo for their postman scripts](https://github.com/cfpb/hmda-platform/tree/master/newman/postman)
* [API scala endpoints](https://github.com/cfpb/hmda-platform/blob/ea0200d47f70108887b2298b52893db32a50312b/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserHttpApi.scala)
    * Scala: "Route =..." defines a route and `pathPrefix` defines what is allowed/required before the route
    * Search for "// GET"
    * /view/nationwide
* [Search their github repository for known filters to discover others](https://github.com/cfpb/hmda-platform)
    * loan_products

### Interesting fields
- "Credit Score of Applicant or Borrower" 


### Filters to consider
* Filter out anything except:
    * Type - Single-family homes
    * Load type - conventional first lien or subsequent
    * Number of applicants: 1 or 2 with 2 representing married couple
    * Purchase of 1 unit (1 home to 1 family)
    * Type of purchaser must not be a business
    * Loan value >= $100,000 and less than $500,000
* Must haves: 
    * Submitted FICO credit scores
        * If only 1 applicant, "Credit Score of Applicant or Borrower" must be between 0 and 1000
        * If 2 applicants, both "Credit Score of Applicant or Borrower" and "Credit Score of Co-Applicant or Co-Borrower" must be between 0 and 1000
    * "Interest Rate" populated and between 0 and 20

### Filters we can use
* [GUI version here](https://ffiec.cfpb.gov/data-browser/data/2018?category=states&items=TX)
* [Hard to find but can search the scala directives for `parameters(`](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)

### Filters you can use
https://ffiec.cfpb.gov/documentation/2018/data-browser-filters/#action_taken

https://github.com/cfpb/hmda-platform/blob/master/docs/v2/UsingModifiedLar.md

---- 

### (Don't use this) Filter mapping
[Link to the scala code for each filter](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala) 

[The models are where you see the mappings](https://github.com/cfpb/hmda-platform/tree/42ba946d21ab20384e2533cde525142d05df2e69/data-browser/src/main/scala/hmda/dataBrowser/models) - use this to map "1" to a categorical

|API filter | json attribute name | LAR schema name | Mapping | What is it? | Line # on scala code |
|--- |--- |--- |--- |--- |--- |
| msamds | | | | [111](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala) |
| years | | | | [125](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
| states | | | | [132](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
| counties | | | | [146](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
| actions_taken | lar: actionTakenType | Action Taken | 6=Purchased | [160](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
| total_units | | | | [196](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
| construction_methods | | | | [233](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
| dwelling_categories | | | | [251](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
| lien_statuses | | | | [269](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
| loan_products | | | 1 - Conventional | [286](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
| loan_purposes | | | | [303](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
|  loan_types| | | | [320](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
|  | | | | [](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
|  | | | | [](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)
|  | | | | [](https://github.com/cfpb/hmda-platform/blob/36efa759dac4bba52c6ea0eb1fd999ca786ef1c3/data-browser/src/main/scala/hmda/dataBrowser/api/DataBrowserDirectives.scala)


In [ ]:
curl -X POST \
  "https://ffiec.cfpb.gov/v2/public/lar/parse" \
  -H 'Content-Type: application/json' \
  -d '{ "lar": "2|10Bx939c5543TqA1144M|10Bx939c5543TqA1144M999143X38|20180721|1|1|1|1|1|110500|1|20180721|123 Main St|Beverly Hills|CA|90210|06037|06037264000|1|1|1|1|1||1|1|1|1|1||3|3|5|7|7|7|7||||5|7|7|7|7||||3|3|1|1|3|3|30|30|36|1|0.428|1|1|750|750|1|9|1|9|10|10|10|10||2399.04|NA|NA|NA|NA|4.125|NA|42.95|80.05|360|NA|1|2|1|1|350500|1|1|5|NA|1|1|12345|1|1|1|1|1||1|1|1|1|1||1|1|1"
}'

{
    "larIdentifier": {
        "id": 2,
        "LEI": "10BX939C5543TQA1144M",
        "NMLSRIdentifier": "12345"
    },
    "loan": {
        "ULI": "10Bx939c5543TqA1144M999143X38",
        "applicationDate": "20180721",
        "loanType": 1,
        "loanPurpose": 1,
        "constructionMethod": 1,
        "occupancy": 1,
        "amount": 110500.0,
        "loanTerm": "360",
        "rateSpread": "0.428",
        "interestRate": "4.125",
        "prepaymentPenaltyTerm": "NA",
        "debtToIncomeRatio": "42.95",
        "combinedLoanToValueRatio": "80.05",
        "introductoryRatePeriod": "NA"
    },
    "larAction": {
        "preapproval": 1,
        "actionTakenType": 1,
        "actionTakenDate": 20180721
    },
    "geography": {
        "street": "123 Main St",
        "city": "Beverly Hills",
        "state": "CA",
        "zipCode": "90210",
        "county": "06037",
        "tract": "06037264000"
    },
    "applicant": {
        "ethnicity": {
            "ethnicity1": 1,
            "ethnicity2": 1,
            "ethnicity3": 1,
            "ethnicity4": 1,
            "ethnicity5": 1,
            "otherHispanicOrLatino": "",
            "ethnicityObserved": 3
        },
        "race": {
            "race1": 5,
            "race2": 7,
            "race3": 7,
            "race4": 7,
            "race5": 7,
            "otherNativeRace": "",
            "otherAsianRace": "",
            "otherPacificIslanderRace": "",
            "raceObserved": 3
        },
        "sex": {
            "sex": 1,
            "sexObserved": 3
        },
        "age": 30,
        "creditScore": 750,
        "creditScoreType": 1,
        "otherCreditScoreModel": "9"
    },
    "coApplicant": {
        "ethnicity": {
            "ethnicity1": 1,
            "ethnicity2": 1,
            "ethnicity3": 1,
            "ethnicity4": 1,
            "ethnicity5": 1,
            "otherHispanicOrLatino": "",
            "ethnicityObserved": 3
        },
        "race": {
            "race1": 5,
            "race2": 7,
            "race3": 7,
            "race4": 7,
            "race5": 7,
            "otherNativeRace": "",
            "otherAsianRace": "",
            "otherPacificIslanderRace": "",
            "raceObserved": 3
        },
        "sex": {
            "sex": 1,
            "sexObserved": 3
        },
        "age": 30,
        "creditScore": 750,
        "creditScoreType": 1,
        "otherCreditScoreModel": "9"
    },
    "income": "36",
    "purchaserType": 1,
    "hoepaStatus": 1,
    "lienStatus": 1,
    "denial": {
        "denialReason1": 10,
        "denialReason2": 10,
        "denialReason3": 10,
        "denialReason4": 10,
        "otherDenialReason": ""
    },
    "loanDisclosure": {
        "totalLoanCosts": "2399.04",
        "totalPointsAndFees": "NA",
        "originationCharges": "NA",
        "discountPoints": "NA",
        "lenderCredits": "NA"
    },
    "nonAmortizingFeatures": {
        "balloonPayment": 1,
        "interestOnlyPayment": 2,
        "negativeAmortization": 1,
        "otherNonAmortizingFeatures": 1
    },
    "property": {
        "propertyValue": "350500.0",
        "manufacturedHomeSecuredProperty": 1,
        "manufacturedHomeLandPropertyInterest": 1,
        "totalUnits": 5,
        "multiFamilyAffordableUnits": "NA"
    },
    "applicationSubmission": 1,
    "payableToInstitution": 1,
    "AUS": {
        "aus1": 1,
        "aus2": 1,
        "aus3": 1,
        "aus4": 1,
        "aus5": 1,
        "otherAUS": ""
    },
    "ausResult": {
        "ausResult1": 1,
        "ausResult2": 1,
        "ausResult3": 1,
        "ausResult4": 1,
        "ausResult5": 1,
        "otherAusResult": ""
    },
    "reverseMortgage": 1,
    "lineOfCredit": 1,
    "businessOrCommercialPurpose": 1
}

In [42]:
baseUrl = "https://ffiec.cfpb.gov/v2/data-browser-api/view/csv?"
    
# Use https://github.com/cfpb/hmda-platform/blob/master/docs/v2/spec/2018_File_Spec_LAR.csv
#    for matching filter codes to what you want
#  
# loan_products = "Loan Type" = loanType
#     1. Conventional (not insured or guaranteed by FHA VA RHS or FSA) 
#     2. Federal Housing Administration insured (FHA)
#     3. Veterans Affairs guaranteed (VA) 
#     4. USDA Rural Housing Service or the Farm Service Agency guaranteed (RHS or FSA)	
# 
# loanPurpose = "Loan Purpose"
#     1. Home purchase 

# lien_statuses = "Lien Status"
#     1. Secured by a first lien 
#     2. Secured by a subordinate lien

# " Type of Purchaser"
#     0. Not applicable
#     1. Fannie Mae
#     2. Ginnie Mae
#     3. Freddie Mac
#     4. Farmer Mac
#     5. Private securitizer
#     6. Commercial bank  savings bank or savings association
#     71. Credit union mortgage company or finance company
#     72.  Life insurance Company
#     8. Affiliate institution 9. Other type of purchaser

# actions_taken
#     1. Loan originated
#     2. Application approved but not accepted
#     3. Application denied
#     4. Application withdrawn by applicant
#     5. File closed for incompleteness
#     6. Purchased loan
#     7. Preapproval request denied
#     8. Preapproval request approved but not accepted

filters = [
    'states=TX'
    , 'years=2018'
    , 'actions_taken=6'
    #, 'loanType=1'
    #, 'loanPurpose=1'
    #, 'loan_products=Conventional:First Lien,Conventional:Subordinate Lien'
    , 'total_units=1'
] 

# Using list comprehensions, create the url filter:
finalUrl = baseUrl + ''.join("&%s" % x for x in filters)
print(finalUrl)

try:    
    response = (requests.get(finalUrl).text)
    time.sleep(0.5)
    print(response)

except Exception as e:
    raise e

https://ffiec.cfpb.gov/v2/data-browser-api/view/csv?&states=TX&years=2018&loan_products=Conventional:First Lien,Conventional:Subordinate Lien


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# FFIEC's HMDA API can return JSON (default) or CSV 
# Can use the "aggregate" endpoint or detail endpoint
# Detail: /view/csv?states=CA,MD,DC&years=2018&actions_taken=5"
# Aggregate: /view/aggregations?states=MD&years=2018&actions_taken=5,6&races=White,Asian,Joint"
r = requests.get('http://api.football-data.org/v1/competitions/398/teams')
x = r.json()
df = pd.DataFrame(x['teams'])
print df
